# Тензоры и базовые операции

В работе напишем класс, который позволят оперировать с тензорами. Тензоры будем хранить в `numpy` массивах. Элементами массива будет объект символьного выражения из `sympy`.
Вообще говоря, это не самый эффективный и быстрый способ вводить тензоры, но удобный из-за того, что `numpy` массивы поддерживают многомерность, и есть универсальный метод 
`numpy.einsum`, позволяющий производить любые свертки данных.

## Системы координат

In [1]:
import numpy as np
import sympy as sp
from IPython.display import Latex, Math
from typing import Callable

Сначала научимся описывать различные криволинейные системы координат на примере сферической и цилиндирической.

Пусть нам дана декартова система координат $(x, y, z)$ с началом в точке $O$.

Определим связь между координатами в декартовой системе координат $(x, y, z)$ и сферической $(r, \theta, \varphi)$,  по формулам
$$
\begin{array}{ccl}
x & = & r \sin\theta \cos\varphi,\\
y & = & r \sin\theta \sin\varphi,\\
z & = & r \cos\theta,
\end{array}
$$
где $\theta$ -- полярный угол, $\varphi$ -- азимутальный угол, $r$ -- радиус точки.

Определим связь между координатами в декартовой системе координат $(x, y, z)$ и цилиндрической $(R, \Phi, Z)$, по формулам
$$
\begin{array}{ccl}
x & = & R \cos\Phi,\\
y & = & R \sin\Phi,\\
z & = & Z,
\end{array}
$$
где $\varphi$ -- азимутальный угол, $r$ -- расстояние до оси $Oz$, $Z$ -- расстояние до плоскости $Oxy$.
 
Введем символы, в которых принято обозначать коордитаны в этих системах координат.

In [2]:
# Декартова
x,y,z = sp.symbols('x y z', real = True)

# Сферическая
r = sp.symbols('r', real = True, nonnegative=True)
theta = sp.symbols('\\theta', real = True, nonnegative=True)
phi = sp.symbols('\\varphi', real = True, nonnegative=True)

# Цилиндрическая
R = sp.symbols('R', real = True, nonnegative=True)
Phi = sp.symbols('\\Phi', real = True, nonnegative=True)
Z = sp.symbols('Z', real = True)

Реализуем класс `CoordinateSystem`, который по заданному преобразованию связи между декартовой системой координат и рассматриваемой умел делать следующие вещи
1. Строить ковариантный и контравариантный базис
2. Строить фундаментальную ковариантную и контравариантную матрицу перехода

Вспомним, что если $\mathbf{r} = x \mathbf{e}_x  + y \mathbf{e}_y + z \mathbf{e}_z$, где 
$$
\begin{array}{clc}
x & = & z(q_1, q_2, q_3),\\  
y & = & y(q_1, q_2, q_3),\\
z & = & z(q_1, q_2, q_3),\\
\end{array}
$$
и $q_i$ -- криволинейные координаты, то ковариантные базис имеет вид
$$
\mathbf{e}_{i} = \frac{\partial \mathbf{r}}{\partial q_i} = 
\frac{\partial x}{\partial q_i} \mathbf{e}_x  + 
\frac{\partial y}{\partial q_i} \mathbf{e}_y + 
\frac{\partial z}{\partial q_i}  \mathbf{e}_z
$$
$$
(i = 1,2,3).
$$

Будем размещать его в матрице `CoordinateSystem.covariant_basis` по столбцам. 

Контравариантный базис $\mathbf{e}^{j}$ имеет свойство 
$$
\mathbf{e}_{i} \cdot \mathbf{e}^{j} = \delta_i^j\quad
(i,j = 1,2,3),
$$
где $\delta_i^j$ -- символ Кронекера. Поэтому матрица `CoordinateSystem.contravariant_basis`, содержащая координаты контравариантного базиса будет просто обратной к `CoordinateSystem.covariant_basis`.

Компоненты фундаментального ковариантного тензора $g_{ij}$ определяются как
$$
    g_{ij} = \mathbf{e}_i \cdot \mathbf{e}_j.
$$

Компоненты фундаментального контравариантного тензора $g^{ij}$ определяются как
$$
    g^{ij} = \mathbf{e}^i \cdot \mathbf{e}^j.
$$


В классе `CoordinateSystem` реализуйте расчет ковариантного и контравариантного базиса, ковариантной и ковариантного фундаментального тензора. 

Манипуляции с матрицами от символьных выражений проще реализовать с помощью `sp.Matrix`. Так как мы храним тензоры в `numpy` массивах, то специально реализована вспомогатльная функции `Matrix_to_numpy` копирования из `sp.Matrix` в `np.ndarray`.

In [3]:
def Matrix_to_numpy(A : sp.Matrix) -> np.ndarray:
    B = np.empty(A.shape, dtype = np.object_)

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            B[i, j] = A[i, j]

    return B

class CoordinateSystem:
    def __init__(
        self, 
        symbols : list,
        direct_transform : Callable,
        inverse_transform : Callable,
        decart_symbols : list = [x, y, z],
        user_covariant_basis = None
    ):
        self.symbols = symbols
        self.decart_symbols = decart_symbols
        
        self.direct_transform = direct_transform
        self.inverse_transform = inverse_transform

        if user_covariant_basis is None:
            coords = self.direct_transform(*self.symbols)
            
            # здесь нужно правильно реализовать вычисления ковариантного базиса
            covariant_basis = sp.Matrix([ ]) # <you code>
        else:
            covariant_basis =  sp.Matrix(user_covariant_basis)

        # здесь нужно правильно реализовать вычисления контравариантного базиса через ковариантный
        contravariant_basis = sp.Matrix([ ]) # <you code>
        
        contravariant_basis.simplify()
        
        self.covariant_basis = Matrix_to_numpy(covariant_basis)
        self.contravariant_basis = Matrix_to_numpy(contravariant_basis)

        # здесь реализовать правильное вычисление контравариантного фундаментального тензора
        g_contr = sp.Matrix([ ]) # <you code>
        g_contr.simplify()

        # здесь реализовать правильное вычисление ковариантного фундаментального тензора
        g_cov = sp.Matrix([ ]) # <you code>
        g_cov.simplify()

        self.g_contr = Matrix_to_numpy(g_contr)
        self.g_cov   = Matrix_to_numpy(g_cov)
        
    def direct_subs(self):
        coords = self.direct_transform(*self.symbols)
        return { k:v for k,v in zip(self.decart_symbols, coords)}

    def inverse_subs(self):
        coords = self.inverse_transform(*self.decart_symbols)
        return { k:v for k,v in zip(self.symbols, coords)}

def cs_subs(
    from_cs : CoordinateSystem,
    to_cs : CoordinateSystem
):
    to_decart = from_cs.inverse_subs()
    to_target = to_cs.direct_subs()

    return { 
        k : v.subs(to_target).simplify() 
        for k, v in to_decart.items()
    }
    
def display_subs(subs : dict):
    for k, v in subs.items():
        display(Math(f"{sp.latex(k)} \\to {sp.latex(v)}"))

Создадим класс, описывающий декартову систему координат

In [4]:
DecartCoordinateSystem = CoordinateSystem(
    [x, y, z],
    lambda x, y, z: (x, y, z),
    lambda x, y, z: (x, y, z)
)

In [94]:
display(sp.Matrix(DecartCoordinateSystem.contravariant_basis))
display(sp.Matrix(DecartCoordinateSystem.covariant_basis))
display(sp.Matrix(DecartCoordinateSystem.g_contr))
display(sp.Matrix(DecartCoordinateSystem.g_cov))
display_subs(DecartCoordinateSystem.direct_subs())
display_subs(DecartCoordinateSystem.inverse_subs())

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

В наших обозначениях, объект, определяющий цилиндрическую систему координат имеет вид.

In [96]:
CylindricalCoordinateSystem = CoordinateSystem(
    [R, Phi, Z],
    lambda R, Phi, Z: (R * sp.cos(Phi), R * sp.sin(Phi), Z),
    lambda x, y, z: (sp.sqrt(x**2+y**2), sp.atan2(y, x), z)
)

display(sp.Matrix(CylindricalCoordinateSystem.contravariant_basis))
display(sp.Matrix(CylindricalCoordinateSystem.covariant_basis))
display(sp.Matrix(CylindricalCoordinateSystem.g_contr))
display(sp.Matrix(CylindricalCoordinateSystem.g_cov))
display_subs(CylindricalCoordinateSystem.direct_subs())
display_subs(CylindricalCoordinateSystem.inverse_subs())

Matrix([
[   cos(\Phi),   sin(\Phi), 0],
[-sin(\Phi)/R, cos(\Phi)/R, 0],
[           0,           0, 1]])

Matrix([
[cos(\Phi), -R*sin(\Phi), 0],
[sin(\Phi),  R*cos(\Phi), 0],
[        0,            0, 1]])

Matrix([
[1,       0, 0],
[0, R**(-2), 0],
[0,       0, 1]])

Matrix([
[1,    0, 0],
[0, R**2, 0],
[0,    0, 1]])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Создайте объект `SphericalCoordinateSystem`, определяющий описание сферической системы координат.

In [ ]:
# <you code>

## Тензоры

Вспомним, что смешанным тензором (например, второго порядка) называется набор из $n^2$ чисел (у нас везде далее $n=3$) $A_{i\cdot}^{\cdot j}$, изменяющийся по закону 
$$
\overline{A}_{i\cdot}^{\cdot j} = A_{\alpha\cdot}^{\cdot \beta} 
\frac{\partial x^\alpha}{\partial \overline{x}^i}
\frac{\partial \overline{x}^j}{\partial x^\beta},
$$
где $\overline{A}_{i\cdot}^{\cdot j}$ -- компоненты тензора в системе координат с чертой, а $x^i = x^i(\overline{x}^j)$ и $\overline{x}^j = \overline{x}^j(x^i)$ -- формулы прямого и обратного преобразования, определяемые системами координат ($i, j = 1,\ldots, n$).

В нашем случае величины $\frac{\partial x^\alpha}{\partial \overline{x}^i}$, $\frac{\partial \overline{x}^j}{\partial x^\beta}$ являются соответствующими компонентами ковариантного и контравариантного базисов, лежащих в матрицах `CoordinateSystem.covariant_basis`, `CoordinateSystem.contravariant_basis`.

Для того, чтобы от контравариантную компоненту тензора превратить в ковариантную, необходимо произвоести свертку тензора, записанного в заданной системе координат с фундаментальным ковариантным тензором этой системы координат
$$
A_{ij} = A^{\alpha\cdot}_{\cdot j} g_{\alpha i}.
$$

Аналогично
$$
A^{ij} = 
A_{\alpha\cdot}^{\cdot j} g^{\alpha i} = 
A^{i}_{\cdot \beta} g^{\beta j}.
$$




Реализуем класс `Tensor`.

Основные требования к классу `Tensor`:
1. Хранение матрицы данных в виде `sympy` выражений.
2. Хранение текущей системы координат.
3. Хранение данных о ковариантных и контравариантных тензорах.
4. Умение опускать и поднимать идексы в текущей системе координат.
5. Умение переводить компоненты тензора в другую систему координат.

Немного про реализацию.
* Хранение матрицы данных будем осуществлять в `numpy.ndarray` `Tensor.data` в виде `sympy` выражений.
* Текущую систему координат будем хранить в переменной `Tensor.cs` в виде объекта класса `CoordinateSystem`.
* Данные о контраваринтных индексах будем хранить и поддерживать в актуальном состоянии в списке `Tensor.contr_idx`. Ковариантнымим индексами будут все остальные.
* Опускание и поднятие индексов будем реализовывать в методах `Tensor.idx_to_cov` и `Tensor.idx_to_contr`, задавая им номер индекса, который необходимо опустить или поднять.
* Метод `Tensor.to` будет переводить тензор из текущей системы координат в заданную.
* Все методы класса проводят модификацию `inplace`, т.е. изменяют поля текущего объекта и возвращают ссылку на него самого.
* Для удобства реализованы методы `Tensor.to_cov` и `Tensor.to_contr`, переводящие все индексы вниз или вверх.
* Для удобства реализован метод `Tensor.simplify`, осуществляющий упрощение всех входящих в тензор выражений.
* Для удобства реализован метод `Tensor.doit`, раскрывающий все производные, входящие в состав тензора.
* Для удобства реализован метод `Tensor.copy`, возвращающий полную копию текущего объекта.
* Для удобства реализован метод `Tensor.subs`, осуществляющий подстановку для всех элементов тензора
* Для удобства реализован метод `Tensor.print` для красивого вывода тензора 



In [118]:
class Tensor:
    subscripts = "jklmnoprstuvabcdefghxyz"
    
    def __init__(
        self,
        data : list,
        cs : CoordinateSystem,
        contr_idx : list = [],
        name = None
    ):
        self.data = np.array(data, dtype = np.object_)
        self.cs = cs
        self.contr_idx = contr_idx
        
        self.order = len(self.data.shape)
        self.n = self.data.shape[0]

        self.name = "A" if name is None else name

    @staticmethod
    def einsum_notation_for_leveling_idx(
        idx : int,
        order : int
    ):
        left_notation = Tensor.subscripts[:idx] + 'i' + Tensor.subscripts[idx+1:order]
        res_notation  = Tensor.subscripts[:idx] + 'q' + Tensor.subscripts[idx+1:order]
        
        return f'{left_notation}, iq -> {res_notation}'

    @staticmethod
    def einsum_notation_for_coordinate_transform(
           order
       ):
        res_notation = Tensor.subscripts[:order]
        left_notation = Tensor.subscripts[order: 2 * order] 

        for idx in range(order):
            left_notation += "," + Tensor.subscripts[order + idx] + Tensor.subscripts[idx]

        return f'{left_notation} -> {res_notation}'

    @staticmethod
    def basis_by_index(index, cov_basis_repr, contr_basis_repr, contr_idx):
        res = ""
        for i, idx in enumerate(index):
            if (i > 0):
                res += "\\otimes"

            if i in contr_idx:
                res += sp.latex(cov_basis_repr[idx])
            else:
                res += sp.latex(contr_basis_repr[idx])
        
        return res
    
    def idx_to_cov(self, idx : int):

        assert 0 <= idx and idx < self.n, f"Индекс должен лежать в диапазоне [0, {self.n})"
        
        if not idx in self.contr_idx:
            return self

        self.contr_idx.remove(idx) # поддержание списка  индексов в актуальном состоянии

        # генерирование нотации для применения np.einsum
        notation = self.einsum_notation_for_leveling_idx(idx, self.order) 
        # Реализуйте свертку и обновление данных с попощью np.einsum и сгенерированной notation
        # self.data = np.einsum(notation, <your code>)

        return self

    def idx_to_contr(self, idx : int):

        assert 0 <= idx and idx < self.n, f"Индекс должен лежать в диапазоне [0, {self.n})"
        
        if idx in self.contr_idx:
            return self

        self.contr_idx.append(idx) # поддержание списка индексов в актуальном состоянии
        
        # генерирование нотации для применения np.einsum
        notation = self.einsum_notation_for_leveling_idx(idx, self.order)
        
        # Реализуйте свертку и обновление данных с попощью np.einsum и сгенерированной notation
        # self.data = np.einsum(notation, <your code>)

        return self

    def to(
        self, 
        cs : CoordinateSystem
    ):
        if (self.cs == cs):
            return self

        # генерирование нотации для применения np.einsum
        notation = self.einsum_notation_for_coordinate_transform(self.order)
        
        # сначала переводим в декартову систему координат        
        g = []
        # необходимо правильно сформировать последовательность производных для корректного 
        # перевода в декартову систему координат
        # необходимо испльзовать self.covariant_basis и self.contravariant_basis
        g_contr = 0 # <your code>
        g_cov = 0 # <your code>
        
        for idx in range(self.order):
            if idx in self.contr_idx:
                g.append(g_cov)
            else:
                g.append(g_contr)

        self.data = np.einsum(notation, self.data, *g)

        # переводим из декартовой в требуемую систему координат        
        g = []

        # необходимо правильно сформировать последовательность производных для корректного 
        # перевода в декартову систему координат
        # необходимо испльзовать cs.covariant_basis и cs.contravariant_basis
        g_contr = # <your code>
        g_cov = # <your code>
    
        for idx in range(self.order):
            if idx in self.contr_idx:
                g.append(g_cov)
            else:
                g.append(g_contr)

        self.data = np.einsum(notation, self.data, *g)

        self.cs = cs
        
        return self

    def to_contr(self):
        contr_idx = self.contr_idx.copy()
        
        for idx in range(self.order):
            self.idx_to_contr(idx)
                
        return self

    def to_cov(self):
        contr_idx = self.contr_idx.copy()
        
        for idx in range(self.order):
            self.idx_to_cov(idx)
                
        return self

    def copy(self):
        return Tensor(self.data.copy(), self.cs, self.contr_idx.copy())
    
    def simplify(self):
        for index, value in np.ndenumerate(self.data):
            self.data[index] = value.simplify()

        return self

    def subs(self, expr : dict):
        for index, value in np.ndenumerate(self.data):
            self.data[index] = self.data[index].subs(expr)
    
        return self

    def doit(self):
        for index, value in np.ndenumerate(self.data):
            self.data[index] = self.data[index].doit()
    
        return self

    def print(self):
        up = ""
        down = ""
        for idx in range(self.order):
            if idx in self.contr_idx:
                up += f"{idx}"
                down += "\\cdot"
            else:
                down += f"{idx}"
                up   += "\\cdot"

        contr_basis_repr = [ sp.symbols(f'\\mathbf{{e}}^{symb}') 
            for symb in self.cs.symbols
        ]

        cov_basis_repr = [ sp.symbols(f'\\mathbf{{e}}_{symb}') 
            for symb in self.cs.symbols
        ]
        
        output = ""

        for index, value in np.ndenumerate(self.data):
            if value != 0:
                if output != "":
                    output += " + "
                output +=  f'\\left({sp.latex(value)}\\right) {self.basis_by_index(index,cov_basis_repr, contr_basis_repr, self.contr_idx )}'
       
        display(Math(f"{self.name}_{{{down}}}^{{{up}}} = {output}"))

## Ковариантные и контравариантные базисные векторы

Реализуйте методы `Tensor.idx_to_contr` и `Tensor.idx_to_contr` для поднимания и опускания индексов. Используйте свертку с соответствующим фундаментальным тензором. Проверьте, что ваш код работает. Для проверки используйте уже введенные декартову, цилиндрическую и сферическую системы координат.

In [ ]:
## <your code>

Известно, что скалярное произведение векторов $\mathbf{a}$ и $\mathbf{b}$ определяется по формуле
$$
\mathbf{a} \cdot \mathbf{b} = a^i b_i,
$$
где $a^i$ -- контравариантные компоненты вектора $\mathbf{a}$, $b_i$ -- ковариантные компоненты вектора $\mathbf{b}$.

Длина вектора определяется как 
$$
l(\mathbf{a}) = \sqrt{\mathbf{a} \cdot \mathbf{a}} = \sqrt{a_i a^i}.
$$

Реализуйте функции для вычисления скалярного произведения вектора `scalar_dot` и длины вектора `l`.

In [ ]:
def scalar_dot(A : Tensor, B : Tensor):
    assert A.data.shape == B.data.shape, "Тензоры должны иметь одну размерность" 
    assert A.order == 1, "Тензор A не является вектором" 
    assert B.order == 1, "Тензор B не является вектором" 

    return # <your code>

def l(A : Tensor):
    assert A.order == 1, "Тензор не является вектором"
    return # <your code>

Далее реализована функция `display_basis_l`, создает вектор, у которого по очереди одна из координат равна `1` и считает её длину.

In [99]:
def display_basis_l(
    cs : CoordinateSystem,
    contr_idx : list = []
):
    for i in range(3):
        v_coords = [sp.Integer(0)]*3
        v_coords[i] = sp.Integer(1)
    
        v = Tensor(v_coords, cs, contr_idx, name = "\\mathbf{v}")
        v.print()
        display(l(v))


Используя эту функцию посчитайте длину всех базисов (ковариантных и ковариантных) для декартовой, цилиндрической и сферической систем координат и получите размерности, числовых значений координат.

In [ ]:
# <your code>

## Единичный тензор или символ Кронекера ($\delta_i^j$)

Реализуйте метод `Tensor.to`, переводящий тензор из текущей системы координат в заданную.

Основные особенности. В реализации `CoordinateSystem` указывается связь между текущей системой координат и декартовой в виде соотношения 
$$
\begin{array}{l}
x = x(q^1, q^2, q^3),\\
y = y(q^1, q^2, q^3),\\
z = z(q^1, q^2, q^3),\\
\end{array}
$$
где $q^i$ -- координаты в текущей криволинейной системе координат ($i=1,2,3$).

Предположим, что нам надо перевести в другую систему координат, описываемую координатами $p^j$ ($j=1,2,3$) задаваемую соотношениями
$$
\begin{array}{l}
x = x(p^1, p^2, p^3),\\
y = y(p^1, p^2, p^3),\\
z = z(p^1, p^2, p^3),\\
\end{array}
$$

Полноправный переход между системами координат $q \to p$ выполняется за два шага
$$
(q^1, q^2, q^3) \quad \to \quad (x,y,z) \quad  \to \quad  (p^1, p^2, p^3).
$$

Для этого необходимо использовать `Tensor.covariant_basis` и `Tensor.contravariant_basis` для соответствующих систем координат.

Для проверки вашего решения используйте символ Кронекера. Его смешанное представление в любой системе координат 
$$
\delta_i^j = \left\{
\begin{array}{ll}
1, & i = j\\
0, & i \neq j.
\end{array}
\right.
$$

Задайте его в декартовой системе координат. Убедитесь, что в ней же его ковариантное и контравариантное представление не меняется. Затем переведите его в цилиндрическую систему координат, посмотрите, как выглядит его ковариантное и контравариантное представление. Затем переведите его в сферическую систему координат, сделайте тоже самое и верните обратно в декуртову систему координат. Если смешанное представление во всех системах координат имеет вид единичной матрицы, а ковариантное и контравариантное -- диагональный вид, скорее всего, вы на правильном пути.

In [ ]:
I = Tensor(
    [[sp.Integer(1), sp.Integer(0), sp.Integer(0)],
     [sp.Integer(0), sp.Integer(1), sp.Integer(0)],
     [sp.Integer(0), sp.Integer(0), sp.Integer(1)]],
    DecartCoordinateSystem,
    [1],
    name = "\\delta"
)
I.print()

I.to(SphericalCoordinateSystem).print()

In [ ]:
# <your code>

## Длина вектора и скалярное произведение векторов

Потестируйте ещё вашу реализацию `Tensor`. Задайте некоторый вектор известной длины и переводя его поочереди в различные системы координат выводите его длину. Она должна быть инвариантом, независимо от системы координат.

In [ ]:
# <your code>

## Градиент ($\nabla f$)

Для функции $f=f(x,y,z)$ градиентом называется вектор
$$
\nabla f = \frac{\partial f}{\partial x} \mathbf{e}_x +
\frac{\partial f}{\partial y} \mathbf{e}_y + 
\frac{\partial f}{\partial z} \mathbf{e}_z.
$$

как известно, $\nabla f$ является ковариантным вектором, т.к. если мы рассмотрим другую систему координат $(q^1, q^2, q^3)$, то для него справедлив закон преобразования ковариантного вектора,
$$
\frac{\partial f}{\partial q_i} = 
\frac{\partial f}{\partial x} \frac{\partial x}{\partial q^i} +
\frac{\partial f}{\partial y} \frac{\partial y}{\partial q^i} +
\frac{\partial f}{\partial z} \frac{\partial z}{\partial q^i}
$$
$$
(i = 1,2,3).
$$

Выведем это соотношение из общих принципов тензорного преобразования вектора и производной.

Реализуйте функцию `grad`, которая по заданной функции `sp.Function` и системе координат возвращает ковариантный вектор из производных функций по её переменным в текущей системе координат.

In [260]:
def grad(
    f : sp.Function,
    cs : CoordinateSystem,
    name = None
):
    return Tensor(
        # <you code>, 
        cs, 
        [], 
        name = name
    )

In [ ]:
fd = sp.Function("f")(x,y,z)
fs = sp.Function("f")(r,theta,phi)
fc = sp.Function("f")(R,Phi,Z)
display(fd)
display(fs)
display(fc)

grad(fd, DecartCoordinateSystem, name = "\\left(\\nabla f\\right)").print()
grad(fs, SphericalCoordinateSystem, name = "\\left(\\nabla f\\right)").print()
grad(fc, CylindricalCoordinateSystem, name = "\\left(\\nabla f\\right)").print()

Сначала необходимо преобразовать вектор градиента из декартовой системы координат в другую, например, цилиндрическую.

In [ ]:
grad_f = grad(fd, DecartCoordinateSystem, name = "\\left(\\nabla f\\right)")
grad_f.print()

grad_f.to(CylindricalCoordinateSystem).print()

Далее необходимо уметь делать подстановку для производной от функции, заданной в одной сиситеме координат через производные в другой системе. Так, например,
$$
f = f(x, y, z) = f(q^1, q^2, q^3),
$$
тогда в выражении градиента $\nabla_x f$, необходимо заменить выражения для производных по переменным $(x,y,z)$ по формулам
$$
\begin{array}{clc}
\frac{\partial f}{\partial x}  & \to &
\frac{\partial f}{\partial q^1} \frac{\partial q^1}{\partial x} +
\frac{\partial f}{\partial q^2} \frac{\partial q^2}{\partial x} +
\frac{\partial f}{\partial q^3} \frac{\partial q^3}{\partial x}, \\
\frac{\partial f}{\partial y}  & \to &
\frac{\partial f}{\partial q^1} \frac{\partial q^1}{\partial y} +
\frac{\partial f}{\partial q^2} \frac{\partial q^2}{\partial y} +
\frac{\partial f}{\partial q^3} \frac{\partial q^3}{\partial y}, \\
\frac{\partial f}{\partial z}  & \to &
\frac{\partial f}{\partial q^1} \frac{\partial q^1}{\partial z} +
\frac{\partial f}{\partial q^2} \frac{\partial q^2}{\partial z} +
\frac{\partial f}{\partial q^3} \frac{\partial q^3}{\partial z}.\\
\end{array}
$$

Реализуйте, функцию `grad_transform_subst`, которая по двум заданным системам координат возвращает подстановку для производных по формулам выше. Учтите, что системы координат в классе `CoordinateSystem` связывают декартову и текущую криволинейную систему координат, поэтому вам необходимо совершить по сути два преобразования, как раньше
$$
(q^1, q^2, q^3) \quad \to \quad 
(x,y,z) \quad \to \quad
(p^1, p^2, p^3).
$$

In [120]:
def grad_transform_subst(
    from_func : sp.Function,
    from_cs : CoordinateSystem,
    to_func : sp.Function,
    to_cs :  CoordinateSystem,
    verbose : bool = True
):
    # сначала описываем матрицу для перевода в декартову систему координат
    from_M = sp.Matrix([
        # <your code>         
    ])
    
    # далее описываем матрицу для перевода из декартовой в требуемую
    to_M = sp.Matrix([
        # <your code>          
    ]).inv()
    
    M = Matrix_to_numpy(from_M * to_M)
   
    to_grad = sp.Matrix([ to_func.diff(symb) for symb in to_cs.symbols ]).transpose()
    from_grad = [ from_func.diff(symb) for symb in from_cs.symbols ]

    res = {var : expr.simplify()  for var, expr in zip(from_grad, to_grad*M)}

    if verbose:
        display_subs(res)
    
    return res

In [ ]:
grad_transform_subst_decart_to_cylindr = grad_transform_subst(
    fd, DecartCoordinateSystem,
    fc, CylindricalCoordinateSystem
)

Используя полученную продстановку, получим градиент функции в цилиндрической системе координат.

In [ ]:
grad_f.subs(grad_transform_subst_decart_to_cylindr).simplify().print()

Рассчитаем длину вектора градиента в цилиндрической системе координат.

In [ ]:
l(grad_f)

Покажите, как преобразуется вектор градиента из декартовой в сферическую систему координат. Посчитайте длину градиента, выведите градиент функции в ковариантной и контравариантной системах координат. 

In [ ]:
# <you code>

# Тензор $dv/dx$ и дивергенция ($\operatorname{div}$)

В механике большой интерес представляет тензор 
$$
\nabla\otimes v = 
\begin{pmatrix}
\frac{\partial v_x}{\partial x} & \frac{\partial v_x}{\partial y} &\frac{\partial v_x}{\partial z} \\
\frac{\partial v_y}{\partial x} & \frac{\partial v_y}{\partial y} &\frac{\partial v_y}{\partial z} \\
\frac{\partial v_z}{\partial x} & \frac{\partial v_z}{\partial y} &\frac{\partial v_z}{\partial z} \\
\end{pmatrix}
$$
для заданного вектора скорости 
$$
\mathbf{v}(x,y,z) = v_x(x,y,z) \mathbf{e}_x +
v_y(x,y,z) \mathbf{e}_y +
v_z(x,y,z) \mathbf{e}_z.
$$

$$
\newcommand{\args}{(r, \theta, \varphi)}
\newcommand{\argd}{(x, y, z)}
$$

Научимся его преобразовывать в другую систему координат. Для этого необходимо совершить три основных шага
1. Уметь преобразовывать вектор $\mathbf{v}$ из декартовой системы координат в криволинейную
    $$
    v_x\argd \mathbf{e}_x +
    v_y\argd \mathbf{e}_y +
    v_z\argd \mathbf{e}_z \quad \to \quad
    v_r\args \mathbf{e}^r +
    v_\theta\args \mathbf{e}^\theta +
    v_\varphi\args \mathbf{e}^\varphi. 
    $$
2. Уметь преобразовывать производные
   $$
   \frac{\partial v_p}{\partial q}\argd \quad (p, q \in \{x, y, z\})
   $$
   через
   $$
   \frac{\partial v_p}{\partial q}\args \quad (p \in \{x, y, z\} , q \in \{r, \theta, \varphi\}).
   $$
3. Уметь преобразовывать тензор $\nabla\otimes v$ в другую криволинейную систему координат.

Введем функции скорости, зависящие как от переменных декартовой системы координат, так и исследуемой криволинейной.

In [192]:
veld = [sp.Function(f"v_{symb}")(x,y,z) for symb in DecartCoordinateSystem.symbols ]
tildaveld = [sp.Function(f"v_{symb}")(r,theta,phi) for symb in DecartCoordinateSystem.symbols ]
vels = [sp.Function(f"v_{symb}")(r,theta,phi) for symb in SphericalCoordinateSystem.symbols ]
tildavels = [sp.Function(f"v_{symb}")(x,y,z) for symb in SphericalCoordinateSystem.symbols ]

In [ ]:
[display(v) for v in veld]
[display(v) for v in tildaveld]
[display(v) for v in vels]
[display(v) for v in tildavels];

Определим тензор $\nabla \otimes \mathbf{v}$ в декартовой системе координат

In [ ]:
dvdx = Tensor(
    [ [veld[i].diff(symb) for symb in [x,y,z]] for i in range(3) ],
    DecartCoordinateSystem,
    [], 
    name = "\\left(\\nabla\\otimes\\mathbf{v}\\right)"       
)
dvdx.print()

Для любого тензора второго порядка его след
$$
\operatorname{tr} A = A_i^i
$$
является инвариантом, его называют первым инвариантом и обозначают $\operatorname{tr} A$ или $I_1$.

Первый инвариант тензора $\nabla \otimes \mathbf{v}$, является дивергенцией вектора $\mathbf{v}$.

Реализуйте функцию свертки `trace` для тензора второго порядка.

In [195]:
def trace(A : Tensor):
    assert A.order == 2, "Ожидается тензор второго порядка"
    B = A.copy()
    # <your code>
    return # your code

In [ ]:
div = trace(dvdx).simplify()
div

Для того, чтобы записать дивергенцию в сферической системе координат сначала заменим аргументы у компонент скорости с $(x,y,z)$ на $(r, \theta, \varphi)$ и пересчитаем производные.

In [ ]:
grad_vel_subs = [ 
    grad_transform_subst(
        vd, DecartCoordinateSystem,
        vs, SphericalCoordinateSystem        
    ) 
    for vd, vs in zip(veld, tildaveld)
]

In [ ]:
div = div.\
    subs(grad_vel_subs[0]).\
    subs(grad_vel_subs[1]).\
    subs(grad_vel_subs[2])
div = div.doit()

$$
\newcommand{\args}{(r, \theta, \varphi)}
\newcommand{\argd}{(x, y, z)}
$$

Получим связь между компонентами $v_r\args, v_{\theta}\args, v_{\varphi}\args$ и $v_x\args, v_y\args, v_z\args$.

In [ ]:
vs = Tensor(vels, SphericalCoordinateSystem, [])
vs.print()

vs.to(DecartCoordinateSystem).print()


In [ ]:
subs = {
    k:v for k,v in zip(tildaveld, vs.data)
}
display_subs(subs)

Совершим полученную подстановку.

In [ ]:
div = div.subs(subs).doit()
div

In [ ]:
div.simplify()

Таким образом, мы получили выражение для дивергенции вектора в сферической системе координат. Сравните с выражением в учебнике. Почему оно отличается или не отличается?

Проделайте эту процедуру для тензора $\nabla\otimes\mathbf{v}$ и получите смешанное представление этого тензора в сферической системе координат. Посчитайте дивергенцию (первый инвариант) в сферической системе координат.

In [ ]:
# <your code>

Получите выражение для тензора $\nabla\otimes\mathbf{v}$ в цилиндрической системе координат и вычислите дивергенцию $\operatorname{div} v$.

In [ ]:
# <your code>

# Тензор напряжений $\sigma$

$$
\sigma = -pI + 2 \mu e,
$$
где $p = p(x,y,z)$ -- давление, $I$ -- единичный тензор, $e$ -- тензор скоростей деформации
$$
    e = \frac{1}{2}\left(
    \nabla v + (\nabla v)^T
    \right)
$$

Реализуйте функции `transpose` транспонирования тензора, `Add` сложения тензоров, `Sub` вычитания тензоров, `scalar_mul` умножения функции на скаляр.

In [247]:
def transpose(A : Tensor):
    assert A.order == 2, "Ожидается тензор второго порядка"
    # не забывайте использовать метод Tensor.copy()
    return # <your code>

def Add(A : Tensor, B : Tensor, name = None):
    assert A.data.shape == B.data.shape, "Тензоры должны иметь одну размерность" 
    assert sorted(A.contr_idx) == sorted(B.contr_idx), "Ожидается одинаковые ковариантные и контравариантные индексы" 
    assert A.cs == B.cs, "Тензоры должны быть заданы в одной системе координат"
    
    # не забывайте использовать метод Tensor.copy()
    return Tensor(
        #<your code>, 
        A.cs, 
        A.contr_idx.copy(), 
        name = name
    )

def Sub(A : Tensor, B : Tensor, name = None):
    assert A.data.shape == B.data.shape, "Тензоры должны иметь одну размерность" 
    assert sorted(A.contr_idx) == sorted(B.contr_idx), "Ожидается одинаковые ковариантные и контравариантные индексы" 
    assert A.cs == B.cs, "Тензоры должны быть заданы в одной системе координат"
    
    # не забывайте использовать метод Tensor.copy()
    return Tensor(
        #<your code>,  
        A.cs, 
        A.contr_idx.copy(), 
        name = name
    )

def scalar_mul(alfa : float, A : Tensor):
    # не забывайте использовать метод Tensor.copy()
    return Tensor(
         #<your code>,
        A.cs, 
        A.contr_idx.copy()
    )

Используя, полученные знания о строении тензора $\nabla\otimes\mathbf{v}$, запишите выражение для тензора $\mathbf{\sigma}$ в цилиндрической и сферической системах координат. Рассмотрите его смешанное представление.

In [248]:
pd = sp.Function("p")(x,y,z)
ps = sp.Function("p")(r,theta,phi)
mu = sp.Symbol("\\mu")

In [ ]:
I = Tensor(
    [[sp.Integer(1), sp.Integer(0), sp.Integer(0)],
     [sp.Integer(0), sp.Integer(1), sp.Integer(0)],
     [sp.Integer(0), sp.Integer(0), sp.Integer(1)]],
    DecartCoordinateSystem,
    [1],
    name = "\\delta"
)
I.print()

In [254]:
sigma = #<your code>

Сравните с выражениями для компонент тензора напряжений для вязкой несжимаемой жидкости в различных системах координат, например, из *Лойцянский Л.Г. Механика жидкости и газа. Учебник для вузов. — 7-е изд., испр. — М.: Дрофа, 2003.*